In [1]:
import os
import csv
import json
import time
import re
import pandas as pd
from openai import AzureOpenAI
from dotenv import load_dotenv

# Load environment variables
load_dotenv("../.env")

# Retrieve credentials from environment variables
azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("OPENAI_API_VERSION")
model_name = os.getenv("AZURE_OPENAI_MODEL_NAME")

# Ensure credentials are loaded
if not azure_endpoint or not api_key or not api_version or not model_name:
    raise ValueError("Missing required environment variables. Please check your .env file.")

# Azure OpenAI configuration
client = AzureOpenAI(
    azure_endpoint=azure_endpoint,
    api_key=api_key,
    api_version=api_version
)

# Path to your CSV file
file_path = "NER_with_sentiment.csv"

# Read the CSV file
df = pd.read_csv(file_path, dtype=str)

# Convert numeric columns, if needed
if "unique_word_count" in df.columns:
    df["unique_word_count"] = pd.to_numeric(df["unique_word_count"], errors="coerce")

# Debug line: Count how many rows already have sentiments
filled_count_start = df["chatgpt4o_sentiments"].notna().sum()
print(f"At the start, we have {filled_count_start} rows with 'chatgpt4o_sentiments' filled in, out of {len(df)} total rows.")

# Filter rows where chatgpt4o_sentiments is missing or NaN
df_missing_sentiment = df[df["chatgpt4o_sentiments"].isna()]

if df_missing_sentiment.empty:
    print("No missing sentiment data to process.")
    exit()

# If unique_word_count exists, optionally sort by it descending
if "unique_word_count" in df_missing_sentiment.columns:
    df_missing_sentiment = df_missing_sentiment.sort_values(
        by="unique_word_count", ascending=False, na_position="last"
    )

# Reset index for batch processing
df_missing_sentiment = df_missing_sentiment.reset_index(drop=True)

def call_openai_api(batch_records):
    """
    Calls the Azure OpenAI API for a given list of row dictionaries (batch_records).
    Returns the parsed JSON list or raises an exception if parsing fails.
    """
    system_prompt = (
        "You are a helpful assistant that analyzes sentiment in Reddit stock comments. "
        "For each row, you are given:\n"
        "  - a unique 'id'\n"
        "  - a comma-separated list of recognized ticker symbols\n"
        "  - the text (ner_text_cleaned)\n\n"
        "For each recognized ticker, classify the sentiment of the text toward that ticker "
        "as one of these options:\n"
        "  - Strongly Positive\n"
        "  - Slightly Positive\n"
        "  - Neutral\n"
        "  - Slightly Negative\n"
        "  - Strongly Negative\n\n"
        "Return a JSON array (no markdown) with one object per row. Each object has this form:\n"
        "{\n"
        '   "id": "<the row id>",\n'
        '   "chatgpt4o_sentiments": {\n'
        '       "<TICKER1>": {"4o": {"label": "<one of the five sentiments>"}},\n'
        '       "<TICKER2>": {"4o": {"label": "<one of the five sentiments>"}},\n'
        '       ...\n'
        "   }\n"
        "}\n\n"
        "If multiple tickers are recognized, include them all as separate keys in the 'chatgpt4o_sentiments' object. "
        "If there are no recognized tickers, return an empty object for 'chatgpt4o_sentiments'. "
        "Do not include any additional commentary."
    )
    
    # Build the user prompt from each row's 'id', 'ner_recognized_tickers', and 'ner_text_cleaned'
    user_prompt = []
    for i, row in enumerate(batch_records):
        identifier = row["id"]  # Unique ID from the CSV
        tickers = row.get("ner_recognized_tickers", "")
        text = row.get("ner_text_cleaned", "")
        user_prompt.append(
            f"{i+1}. ID: {identifier} | Tickers: {tickers} | Text: {text}"
        )

    joined_comments = "\n".join(user_prompt)

    messages = [
        {"role": "system", "content": system_prompt},
        {
            "role": "user",
            "content": (
                "Please analyze the following rows:\n"
                f"{joined_comments}\n"
            )
        }
    ]

    response = client.chat.completions.create(
        model=model_name,
        messages=messages,
        max_tokens=2000,
        temperature=0
    )

    if not response or not response.choices:
        raise ValueError("Empty or invalid response from OpenAI API.")

    content = response.choices[0].message.content.strip()
    # Remove potential triple-backtick fences
    content = re.sub(r"^```(json|javascript)\n|\n```$", "", content.strip())
    # Parse JSON
    return json.loads(content)

def process_batch(batch_records, df):
    """
    Attempts to process a list of row dictionaries (batch_records).
    If JSON decoding fails (likely truncation), it splits the batch into smaller pieces.
    """
    if not batch_records:
        return

    try:
        results = call_openai_api(batch_records)
    except json.JSONDecodeError as e:
        # Likely truncated JSON. If batch has more than 1 row, split and retry.
        if len(batch_records) > 1:
            half = len(batch_records) // 2
            batch_part1 = batch_records[:half]
            batch_part2 = batch_records[half:]
            process_batch(batch_part1, df)
            process_batch(batch_part2, df)
        else:
            # Single row but still fails. Skip it.
            row_id = batch_records[0]["id"]
            print(f"JSON decode error on single row (id={row_id}), skipping. Error: {e}")
        return
    except Exception as e:
        print(f"Error calling OpenAI API on batch of size {len(batch_records)}. Error: {e}")
        return

    # If the response is not a list, we can't proceed
    if not isinstance(results, list):
        print("Response was not a JSON list. Skipping this batch.")
        return

    # Update DataFrame for each returned row
    for result in results:
        row_id = str(result.get("id"))
        sentiments_obj = result.get("chatgpt4o_sentiments", {})
        # Convert object to JSON string
        sentiments_str = json.dumps(sentiments_obj)
        df.loc[df["id"] == row_id, "chatgpt4o_sentiments"] = sentiments_str
    
    # Save CSV after successfully updating these rows
    df.to_csv(file_path, index=False)
    print(f"Processed and saved {len(results)} rows.")

###################
# MAIN SCRIPT LOGIC
###################

batch_size = 20  # Adjust as needed

num_missing = len(df_missing_sentiment)
num_batches = (num_missing // batch_size) + (1 if num_missing % batch_size else 0)

for i in range(num_batches):
    start_idx = i * batch_size
    end_idx = start_idx + batch_size
    batch_df = df_missing_sentiment.iloc[start_idx:end_idx]
    
    batch_records = batch_df.to_dict(orient="records")
    print(f"\n=== Processing batch {i+1}/{num_batches} (rows {start_idx} to {end_idx-1}) ===")
    process_batch(batch_records, df)
    
    # After processing each batch, count how many are filled
    filled_count_after_batch = df["chatgpt4o_sentiments"].notna().sum()
    print(f"After batch {i+1}, we have {filled_count_after_batch} rows with 'chatgpt4o_sentiments' filled in.")
    
    # Optional small pause to avoid rate limiting
    time.sleep(1)

print("Sentiment analysis completed.")


At the start, we have 1034 rows with 'chatgpt4o_sentiments' filled in, out of 249222 total rows.

=== Processing batch 1/12410 (rows 0 to 19) ===
Processed and saved 19 rows.
After batch 1, we have 1053 rows with 'chatgpt4o_sentiments' filled in.

=== Processing batch 2/12410 (rows 20 to 39) ===
Processed and saved 20 rows.
After batch 2, we have 1073 rows with 'chatgpt4o_sentiments' filled in.

=== Processing batch 3/12410 (rows 40 to 59) ===
Processed and saved 19 rows.
After batch 3, we have 1092 rows with 'chatgpt4o_sentiments' filled in.

=== Processing batch 4/12410 (rows 60 to 79) ===
Processed and saved 20 rows.
After batch 4, we have 1112 rows with 'chatgpt4o_sentiments' filled in.

=== Processing batch 5/12410 (rows 80 to 99) ===
Processed and saved 20 rows.
After batch 5, we have 1132 rows with 'chatgpt4o_sentiments' filled in.

=== Processing batch 6/12410 (rows 100 to 119) ===
Processed and saved 20 rows.
After batch 6, we have 1152 rows with 'chatgpt4o_sentiments' filled 

KeyboardInterrupt: 